In [1]:
import pandas as pd
import openpyxl
from setup_data import tasas, franquicia, via_importacion, tipo_vhl, seg_casco, seg_rc, seg_ovp

In [2]:
file_path = "listaod especial.xlsx"

df = pd.read_excel(file_path)

In [3]:
def add_new_column(df):
    df["Cobertura Básica / Casco"] = df["Auto Cober. Básica 1"] + df["Auto Cober. Básica 2"] + df["Auto Accesorios"]
    return df

In [4]:
def round_to_million(n):
    if n > 150000000:
        return 150000000
    else:
        return (n // 1000000) * 1000000

In [5]:
def add_capital_seg_column(df):
    df["Capital seg"] = df["Cobertura Básica / Casco"].apply(round_to_million)
    return df

In [6]:
def add_resp_civil_column(df):
    df["Resp. Civil"] = df["Auto Lesión 2 o más Per."] + df["Auto Daños Material a Ter."]
    return df

In [7]:
def add_ovp_column(df):
    df["OVP"] = (df["Auto Muerte/Incap."] + df["Auto Asist. Médica"]) *  df["Cant. Ocupantes"]
    return df

In [8]:
def add_franquicia_column(df):
    df["Franquicia"] = df["Franquicia Monto Fijo"].map(franquicia)
    return df

In [9]:
def add_importacion_column(df):
    df["Importacion"] = df["Via Importación"].map(via_importacion)
    return df

In [10]:
def add_tipo_column(df):
    df['Tipo'] = df['Tipo Vehiculo'].map(tipo_vhl)
    return df

In [11]:
def add_segmento_column(df):
    df['Segmento'] = df['Tipo'] + df['Franquicia'] + df['Importacion']
    return df

In [12]:
def add_segmento_casco_column(df):
    df['Segmento Casco'] = df['Segmento'].map(seg_casco)
    return df

def add_segmento_rc_column(df):
    df['Segmento RC'] = df['Segmento'].map(seg_rc)
    return df

def add_segmento_ovp_column(df):
    df['Segmento OVP'] = df['Segmento'].map(seg_ovp)
    return df

In [13]:
df = add_new_column(df)
df = add_capital_seg_column(df)
df = add_resp_civil_column(df)
df = add_ovp_column(df)
df = add_tipo_column(df)
df = add_franquicia_column(df)
df = add_importacion_column(df)
df = add_segmento_column(df)
df = add_segmento_casco_column(df)
df = add_segmento_rc_column(df)
df = add_segmento_ovp_column(df)
print(df)

     Emitido  Sección Principal Nombre Sección Principal  Sección  \
0          A                500              AUTOMOVILES      501   
1          A                500              AUTOMOVILES      501   
2          A                500              AUTOMOVILES      501   
3          A                500              AUTOMOVILES      501   
4          A                500              AUTOMOVILES      501   
...      ...                ...                      ...      ...   
6279       N                500              AUTOMOVILES      501   
6280       N                500              AUTOMOVILES      501   
6281       N                500              AUTOMOVILES      501   
6282       N                500              AUTOMOVILES      501   
6283       N                500              AUTOMOVILES      501   

     Nombre Sección  Póliza  Endoso  Tipo Póliza Nombre Tipo Póliza  \
0       AUTOMOVILES     551       0            2         Renovacion   
1       AUTOMOVILES     552  